In [1]:
import sys
sys.path.append('/cellar/users/zkoch/methylation_and_mutation/source_files')
%load_ext autoreload
%aimport mutation_features, methylation_pred, somatic_mut_clock, methylation_motifs
%autoreload 1
import get_data, utils, mutation_features, methylation_pred, somatic_mut_clock

In [ ]:
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import dask.dataframe as dd
from scipy.stats import spearmanr, pearsonr
from scipy import stats
import dask
import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV
import xgboost as xgb
import shap
from pymemesuite.common import MotifFile
import Bio.SeqIO
from pymemesuite.common import Sequence
from pymemesuite.fimo import FIMO


%config InlineBackend.figure_format = 'retina'
plt.style.use("seaborn-deep")
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.


# Read ICGC

In [3]:
# qnorm 
icgc_mut_w_age_df, illumina_cpg_locs_df, icgc_methyl_age_df_t, icgc_matrix_qtl_dir, icgc_covariate_fn = get_data.read_icgc_data()

reading in data
Got mutations and metadata, reading methylation
Got methylation, transposing
Done


# ICGC GPU test

In [4]:
motif_occurence_df = pd.read_parquet(
        "/cellar/users/zkoch/methylation_and_mutation/data/methylation_motifs_weiWang/motif_occurences/motif_occurences_combined_15kb.parquet"
        )
mut_feat_icgc = mutation_features.mutationFeatures(
        all_mut_w_age_df = icgc_mut_w_age_df, illumina_cpg_locs_df = illumina_cpg_locs_df, 
        all_methyl_age_df_t = icgc_methyl_age_df_t, out_dir = "./", 
        consortium = 'ICGC', dataset = '', cross_val_num = 0,
        matrix_qtl_dir = icgc_matrix_qtl_dir,
        covariate_fn = icgc_covariate_fn, motif_occurence_df = motif_occurence_df
    )

The least populated class in y has only 1 members, which is less than n_splits=5.


In [5]:
mut_feat_params = {
    'aggregate': 'Both',
    'num_correl_sites': 500,
    'max_meqtl_sites': 0,
    'nearby_window_size': 50000,
    'extend_amount': 500
}

mut_feat_icgc.create_all_feat_mats(
    cpg_ids = icgc_methyl_age_df_t.columns[7:8].tolist(), 
    aggregate = mut_feat_params['aggregate'],
    num_correl_sites = mut_feat_params['num_correl_sites'], 
    max_meqtl_sites=mut_feat_params['max_meqtl_sites'],
    nearby_window_size = mut_feat_params['nearby_window_size'], 
    extend_amount = mut_feat_params['extend_amount'] 
    )

Finished 0 of 2


In [7]:
methyl_pred = methylation_pred.methylationPrediction(
                mut_feat_store = mut_feat_icgc.mutation_features_store,
                #mut_feat_store_fns = mut_feat_store_fns,
                model_type = "xgboost",
                baseline = "none",
                agg_only = True,
                scale_counts_within_dataset = True,
                predict_with_random_feat = -1,
                illumina_cpg_locs_df = illumina_cpg_locs_df,
                all_methyl_age_df_t = icgc_methyl_age_df_t
                )
methyl_pred.train_all_models()
methyl_pred.apply_all_models()
methyl_pred.calc_performance_by_dataset()


OUTER: done training 0 CpGs of 2
Predicted methylation for 0 CpGs of 2
['PRAD', 'PACA', 'PBCA', 'CLLE', 'PAEN', 'UTCA']
Done getting performance of dataset: PRAD
Done getting performance of dataset: PACA
Done getting performance of dataset: PBCA
Done getting performance of dataset: CLLE
Done getting performance of dataset: PAEN
Done getting performance of dataset: UTCA


In [17]:
methyl_pred.save_models_and_preds(out_dir = "/cellar/users/zkoch/methylation_and_mutation/test_out")

wrote out trained models and predictions to /cellar/users/zkoch/methylation_and_mutation/test_out
wrote out performance by dataset to /cellar/users/zkoch/methylation_and_mutation/test_out


In [18]:
test_df = pd.read_parquet("/cellar/users/zkoch/methylation_and_mutation/test_out/performance_by_dataset_xgboost_nonebaseline_agg_only_predict_with_random_feat_2.parquet")